In [ ]:
# IMPORTS
import dotenv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
from pathlib import Path

# RELATIVE
import plots as pl
from analysis_utils import print_metrics, read_time_series_from_file

In [ ]:
# LOAD ENVIRONMENT VARIABLES
dotenv.load_dotenv("../config/local.env")

In [ ]:
# GENERAL SETTINGS
path_res = Path(os.environ["PATH_SIM_RESULTS"])  # results directory
path_output = Path(os.environ["PATH_ANALYSIS_RESULTS"])  # output directory
file_res = "df_evaluation.csv"  # file name evaluation results
flag_cust_res = True  # if True, the number of unblocked devices is included in the time series plots
list_metrics = ["sum_p_tot_sq_eval", "max_p_tot_eval", "cost_1_eval"]  # which metrics to print
colors_single_days = ["red", # inflexible load
                      "green80",  # no control case
                      "blue"]  # RL agent
colors_single_days = [pl.utils.dict_colors[c] for c in colors_single_days]
alpha_mean_daily = 0.9  # transparency of the lines in the mean daily plots
tz_plots = "Europe/Zurich"  # time zone for the plots

# WINTER

In [ ]:
# PATHS
# Specify the directory names with the simulation results for the different cases
dict_res_w = {  # Note: the order matters; put RL case last such that it is in the foreground
    "prop": {
        "dir": "240310_150456_prop_infl_per_day_dynamic",
        "label": "Proportional price",
        "color": "bronze"
    },
    "none": {
        "dir": "240319_132440_none_none_winter",
        "label": "No control",
        "color": "green80"
    },
    "rl": {
        "dir": "240304_201013_rl_dynamic/final_eval",  # use /, not \
        "label": "RL agent",
        "color": "blue"
    },
}
# Reformat to dictionary with dataframes, needed for some plot functions
dict_data_w = {}
for k, v in dict_res_w.items():
    df = read_time_series_from_file(path_res / v["dir"] / file_res, tz=tz_plots)
    dict_data_w[v["label"]] = (df, pl.dict_colors[v["color"]])
# Add inflexible load at the beginning of the dictionary
df_infl = dict_data_w["RL agent"][0].copy().loc[:, ["p_infl"]].rename(columns={"p_infl": "p_tot"})
dict_data_w = {"$P^{\mathrm{infl}}$": (df_infl, pl.dict_colors["red"]), **dict_data_w}

In [ ]:
# METRICS
print_metrics(path_res=path_res, dict_res=dict_res_w, list_metrics=list_metrics)

In [ ]:
# PLOT TIME SERIES
with plt.rc_context(pl.params.single_days):
    # Extreme days
    _ = pl.plot_single_days(path_res=path_res,
                            dict_res=dict_res_w,
                            path_output=path_output,
                            tz_plots=tz_plots,
                            flag_cust_res=flag_cust_res,
                            list_colors=colors_single_days,
                            lw_legend_frame=0.2)
    # Flat price profile
    list_days = [(23, 2022, "flat_price")]
    _ = pl.plot_single_days(path_res=path_res,
                            dict_res=dict_res_w,
                            path_output=path_output,
                            tz_plots=tz_plots,
                            flag_cust_res=flag_cust_res,
                            list_colors=colors_single_days,
                            lw_legend_frame=0.2,
                            list_days=list_days)

In [ ]:
# PLOT DURATION CURVE
with plt.rc_context(pl.params.duration):
    _ = pl.plot_multiple_duration_curves(dict_data=dict_data_w,
                                         col="p_tot",
                                         flag_axins_max=True,
                                         axins_max=[0.5, 0.62, 0.48, 0.32],
                                         axins_max_cords=[-0.05, 0.5, 75, 135],
                                         axins_x_loc=0.2,
                                         axins_y_loc=25,
                                         figsize=(3.3, 1),
                                         path_output=path_output / f"{dict_res_w['rl']['dir'].split('/')[0]}_duration.png"
                                         )

In [ ]:
# PLOT MEAN DAILY PROFILES
list_cols = [["p_tot", "$P^{\mathrm{tot}}$"],
             ["p_wh", "$P^{\mathrm{EWH}}$"],
             ["p_hp", "$P^{\mathrm{HP}}$"],
             ["p_ev", "$P^{\mathrm{EV}}$"]]
with plt.rc_context(pl.params.mean_daily):
    _ = pl.plot_mean_daily(dict_data=dict_data_w,
                           list_cols=list_cols,
                           fontsize_ylabel=7,
                           alpha=alpha_mean_daily,
                           args_legend={"loc": "center",
                                        "bbox_to_anchor": (0.45, 0.98),
                                        "ncol": 4,
                                        "columnspacing":0.64,
                                        "borderpad": 0.3
                                        },
                           figsize=(2.15, 1.1),
                           path_output=path_output / f"{dict_res_w['rl']['dir'].split('/')[0]}_mean_daily.png"
                           )

In [ ]:
# HEATMAP PRICE
with plt.rc_context(pl.params.heatmap):
    fig = pl.HeatmapFigure(series=dict_data_w["RL agent"][0].loc[:, "price_n"],
                           figsize=(1, 1.1),
                           flag_histx=False,
                           flag_histy=False,
                           cbar_label="",
                           annotate_suntimes=False,
                           )
    # Adjustments
    fig.ax_heatmap.xaxis.set_major_formatter(mdates.DateFormatter("%b '%y"))
    fig.ax_heatmap.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    fig.ax_heatmap.xaxis.set_minor_locator(mdates.MonthLocator())
    fig.savefig(path_output / f"{dict_res_w['rl']['dir'].split('/')[0]}_heatmap_price_n.png")

# SUMMER

In [ ]:
# PATHS
dict_res_s = {  # Note: the order matters; put RL case last such that it is in the foreground
    "prop": {
        "dir": "240311_072240_prop_infl_per_day_dynamic",
        "label": "Proportional price",
        "color": "bronze"
    },
    "none": {
        "dir": "240319_132617_none_none_summer",
        "label": "No control",
        "color": "green80"
    },
    "rl": {
        "dir": "240308_203551_rl_dynamic/final_eval",  # use /, not \
        "label": "RL agent",
        "color": "blue"
    }
}
# Reformat to dictionary with dataframes, needed for some plot functions
dict_data_s = {}
for k, v in dict_res_s.items():
    df = read_time_series_from_file(path_res / v["dir"] / file_res, tz=tz_plots)
    dict_data_s[v["label"]] = (df, pl.dict_colors[v["color"]])
# Add inflexible load at the beginning of the dictionary
df_infl = dict_data_s["RL agent"][0].copy().loc[:, ["p_infl"]].rename(columns={"p_infl": "p_tot"})
dict_data_s = {"$P^{\mathrm{infl}}$": (df_infl, pl.dict_colors["red"]), **dict_data_s}

In [ ]:
# METRICS
print_metrics(path_res=path_res, dict_res=dict_res_s, list_metrics=list_metrics)

In [ ]:
# PLOT TIME SERIES
with plt.rc_context(pl.params.single_days):
    # Extreme days
    _ = pl.plot_single_days(path_res=path_res,
                            dict_res=dict_res_s,
                            path_output=path_output,
                            tz_plots=tz_plots,
                            flag_cust_res=flag_cust_res,
                            flag_hp=False,
                            list_colors=colors_single_days,
                            lw_legend_frame=0.2)

In [ ]:
# PLOT DURATION CURVE
with plt.rc_context(pl.params.duration):
    fig, ax = pl.plot_multiple_duration_curves(dict_data=dict_data_s,
                                               col="p_tot",
                                               flag_axins_max=True,
                                               flag_axins_min=True,
                                               axins_max=[0.5, 0.7, 0.48, 0.25],
                                               axins_min=[0.1, 0.17, 0.48, 0.25],
                                               axins_max_cords=[-0.05, 0.5, 40, 80],
                                               axins_min_cords=[99.5, 100.05, -30, -20],
                                               axins_x_loc=0.2,
                                               axins_y_loc=20,
                                               ylim=(-40, 75),
                                               figsize=(3.3, 1.2), # a bit higher because of additional inset
                                               path_output=path_output / f"{dict_res_s['rl']['dir'].split('/')[0]}_duration.png"
                                               )

In [ ]:
# PLOT MEAN DAILY PROFILES
list_cols = [["p_tot", "$P^{\mathrm{tot}}$"],
             ["p_wh", "$P^{\mathrm{EWH}}$"],
             ["p_ev", "$P^{\mathrm{EV}}$"]]
with plt.rc_context(pl.params.mean_daily):
    _ = pl.plot_mean_daily(dict_data=dict_data_s,
                           list_cols=list_cols,
                           fontsize_ylabel=7,
                           alpha=alpha_mean_daily,
                           args_legend={"loc": "center",
                                        "bbox_to_anchor": (0.45, 0.98),
                                        "ncol": 4,
                                        "columnspacing": 0.64,
                                        "borderpad": 0.3
                                        },
                           figsize=(2.15, 1.1),
                           path_output=path_output / f"{dict_res_s['rl']['dir'].split('/')[0]}_mean_daily.png"
                           )

In [ ]:
# HEATMAP PRICE
with plt.rc_context(pl.params.heatmap):
    fig = pl.HeatmapFigure(series=dict_data_s["RL agent"][0].loc[:, "price_n"],
                           figsize=(1, 1.1),
                           flag_histx=False,
                           flag_histy=False,
                           cbar_label="",
                           annotate_suntimes=False,
                           )
    # Adjustments
    fig.ax_heatmap.xaxis.set_major_formatter(mdates.DateFormatter("%b '%y"))
    fig.ax_heatmap.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    fig.ax_heatmap.xaxis.set_minor_locator(mdates.MonthLocator())
    fig.savefig(path_output / f"{dict_res_s['rl']['dir'].split('/')[0]}_heatmap_price_n.png")